In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
from jubileedemo import meatspace_protocols as meat
from jubileedemo import user_functions
import numpy as np
from bayesopt import acquisitions 
import json

import cv2

import bayesopt.bayesian_optimizer as bayesian_optimizer
import bayesopt.model as model
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels

In [3]:
from jubileedemo import image_processing as img

### Connect to robot

First, we connect to the robot by instantiating a jubilee object from the jubilee library. We haven't talked about objects in python or how we work with them. You can think of the jubilee object as a virtual model of the actual robot that keeps tracks of things like which tool the robot is using and knows what things the robot can do. 

In [5]:
jub = meat.BayesianOptDemoDriver(deck_config_filepath = 'deck_config.json')

Loading deck config file from deck_config.json


We need to set a height for the bed so that the tools won't crash into anything while the robot moves

In [6]:
jub.safe_z = None

If the liquid lines are not filled already, fill them

In [7]:
# prime pumps
jub.prime_lines('2.A2', volume = 35)


# Get 12 points of random data

In [7]:
sample_space = user_functions.get_constrained_points(101)

In [8]:
n_init_points = 12
sampled_indices = np.random.randint(0, len(sample_space), size = n_init_points)
sampled_points = sample_space[sampled_indices]

In [9]:
rgb_vals = []
for i in range(len(sampled_points)):
    point = sampled_points[i,:]
    well = jub.next_sample_well()
    print('creating sample')
    print(point)
    print(well)
    
    RGB = user_functions.sample_point(jub, point, 5, well)
    rgb_vals.append(RGB)

creating sample
[0.08 0.67 0.25]
3.A1


TypeError: sample_point() missing 1 required positional argument: 'file_name'

In [34]:
loss_vals = [user_functions.color_loss_calculation(target_color, user_functions.normalize_color(rgb)) for rgb in rgb_vals]

In [44]:
init_data = []
for i in range(len(sampled_points)):
    data = {}
    data['sample_id'] = str(i)
    data['RYB_point'] = list(sampled_points[i])
    data['RGB_measured'] = rgb_vals[i]

    init_data.append(data)
    

In [45]:
with open('initial_data_random_12.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

# BO campaign

## 1. Pick a color

Pick an RGB color that you would like to make

In [8]:
target_color = (178, 255, 102) #peachy
target_color = user_functions.normalize_color(target_color)

## 2. Load some initial data for the optimizer to use

In [9]:
init_data = []

with open('initial_data_random_12.jsonl', 'rt') as f:
    for line in f:
        init_data.append(json.loads(line))

In [10]:
init_X = np.zeros((len(init_data),3))
init_y = np.zeros(len(init_data))

for i, entry in enumerate(init_data):
    init_X[i,:] = entry['RYB_point']
    init_y[i] = user_functions.color_loss_calculation(target_color, user_functions.normalize_color(entry['RGB_measured']))
    

## 3. Pick an acquisition function to use for the optimization:

In [11]:
acquisition_function = acquisitions.optimize_EI

## 4. Prepare final things and run the campaign

In [29]:
initial_data = (init_X, init_y)
number_of_iterations = 3
jubilee = jub

In [28]:
jub.clear_well_usage()

In [30]:
bo_obj, ryb_sampled, rgb_measured = user_functions.BO_campaign(initial_data, acquisition_function, number_of_iterations, target_color, jubilee)

first query pt:  [0. 1. 0.]
Starting iteration 0
Dispensing into well 3.A1
RYB values tested: [0. 1. 0.]
RYB after norm in sample func:  [0.0, 1.0, 0.0]
Dispensing volumes:  [0.0, 5.0, 0.0]
current active tool:  1
swapping
loss score: 0.6856808850238488
RGB values observed: [0.8822679613591773, 0.7851545258001225, 0.2632604756370362]
Starting iteration 1
Dispensing into well 3.A2
RYB values tested: [0. 0. 1.]
RYB after norm in sample func:  [0.0, 0.0, 1.0]
Dispensing volumes:  [0.0, 0.0, 5.0]
current active tool:  1
swapping
loss score: 0.2792119023496661
RGB values observed: [0.11826391232082381, 0.6977397868006349, 0.7033724490433522]
Starting iteration 2
Dispensing into well 3.A3
RYB values tested: [0.  0.8 0.2]
RYB after norm in sample func:  [0.0, 0.8, 0.2]
Dispensing volumes:  [0.0, 4.0, 1.0]
current active tool:  1
swapping
loss score: 0.3335867634485471
RGB values observed: [0.1681470650720454, 0.6457151426536197, 0.5944303227983354]


In [29]:
init_data

[{'sample_id': '0',
  'RYB_point': [0.37, 0.49, 0.14],
  'RGB_measured': [101.51293817718292,
   103.28718929254302,
   172.56010197578075]},
 {'sample_id': '1',
  'RYB_point': [0.72, 0.14, 0.14],
  'RGB_measured': [133.88859145952836, 95.62651370299554, 181.11612492033143]},
 {'sample_id': '2',
  'RYB_point': [0.35000000000000003, 0.61, 0.04],
  'RGB_measured': [115.27877629063097, 136.3812619502868, 200.83671128107073]},
 {'sample_id': '3',
  'RYB_point': [0.55, 0.2, 0.25],
  'RGB_measured': [125.35436583811345, 92.0970044614404, 144.84537922243467]},
 {'sample_id': '4',
  'RYB_point': [0.27, 0.33, 0.4],
  'RGB_measured': [128.74225621414914,
   108.36583811344805,
   151.20038240917782]},
 {'sample_id': '5',
  'RYB_point': [0.25, 0.25, 0.5],
  'RGB_measured': [184.46513702995537, 166.2295729764181, 126.10325047801147]},
 {'sample_id': '6',
  'RYB_point': [0.26, 0.12, 0.62],
  'RGB_measured': [183.86271510516252,
   148.26602931803697,
   114.18891013384321]},
 {'sample_id': '7',
  '

In [31]:
with open('initial_data_24pts_combine.jsonl', 'wt') as f:
    for entry in init_data:
        f.write(json.dumps(entry) + '\n')

In [65]:
#image = cv2.imread('sampleimage_masked_1687813525.8915021.jpg')
image = cv2.imread('sampleimage_masked_1687813298.037668.jpg')

In [71]:
from jubileedemo import image_processing

In [72]:
rgb = image_processing._get_rgb_avg(image)

swapping


In [73]:
norm_rgb = user_functions.normalize_color(rgb)

In [74]:
user_functions.color_loss_calculation(target_color, norm_rgb)

0.2617500255474996

In [75]:
norm_rgb

[0.6621086280218055, 0.19737187036334725, 0.28539823707644374]

In [76]:
target_color

[0.5333333333333333, 0.011764705882352941, 0.9882352941176471]

In [67]:
cv2.imshow('test', image)
cv2.waitKey(0)   #wait for a keyboard input
cv2.destroyAllWindows()

In [68]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [34]:
with open('sample_rgb_values.pkl', 'rb') as f:
    test_rgb = pickle.load(f)

In [13]:
image = jubilee.well_image('1.A1')

current active tool:  1


In [31]:
cv2.imdecode(image)

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'imdecode'
> Overload resolution failed:
>  - imdecode() missing required argument 'flags' (pos 2)
>  - imdecode() missing required argument 'flags' (pos 2)


In [14]:
imgbuf = np.frombuffer(image, dtype = np.uint8)

In [15]:
imgcv = cv2.imdecode(imgbuf, cv2.IMREAD_COLOR)

In [17]:
imgcv[:,:,[2,1,0]]

array([[[107,  97, 105],
        [108,  98, 106],
        [109,  99, 107],
        ...,
        [106,  95,  99],
        [104,  93,  97],
        [103,  92,  96]],

       [[108,  98, 106],
        [109,  99, 107],
        [110, 100, 108],
        ...,
        [106,  95,  99],
        [104,  93,  97],
        [103,  92,  96]],

       [[110, 100, 108],
        [111, 101, 109],
        [112, 102, 110],
        ...,
        [106,  95,  99],
        [104,  93,  97],
        [103,  92,  96]],

       ...,

       [[ 63,  58,  64],
        [ 63,  58,  64],
        [ 63,  58,  64],
        ...,
        [102,  92,  90],
        [101,  91,  90],
        [100,  90,  89]],

       [[ 63,  58,  64],
        [ 63,  58,  64],
        [ 63,  58,  64],
        ...,
        [102,  92,  90],
        [101,  91,  90],
        [100,  90,  89]],

       [[ 63,  58,  64],
        [ 63,  58,  64],
        [ 63,  58,  64],
        ...,
        [102,  92,  90],
        [101,  91,  90],
        [100,  90,  89]]

In [41]:
import matplotlib.pyplot as plt

In [43]:
plt.imshow(imgcv)
plt.show()

In [37]:
imgarr = imgbuf.reshape(600, 600, 3)

ValueError: cannot reshape array of size 12799 into shape (600,600,3)

In [40]:
loss_vals = []
for entry in test_rgb:
    loss = user_functions.color_loss_calculation(entry, target_color)
    loss_vals.append(loss)
    

In [41]:
import matplotlib.pyplot as plt

In [44]:
plt.plot(loss_vals)